In [1]:
from dgl.data.utils import save_graphs
import dgl
import torch as th
import scipy.sparse as sp
import networkx as nx
import pickle
from numpy import array
import pandas as pd

#Aprire i File
import os
#Per la gestione dei file già esistenti
from os.path import exists

#DataFrame
import pandas as pd

#Rimozione duplicati lista
from collections import OrderedDict

#Aprire i File
import os
#Per la gestione dei file già esistenti
from os.path import exists

#Salvataggio dizinario
import pickle 

from sklearn import preprocessing


Using backend: pytorch


In [2]:
proteine_path = "../Dati/proteine.csv"
proteine = pd.read_csv(proteine_path, sep = ',')

Amino = []
ID_Graph = []
for filename in os.listdir("../../R/Rdati/Matrici Adiacenza/"):
    with open(os.path.join("../../R/Rdati/Matrici Adiacenza/", filename), 'r', encoding='windows-1252') as f:
        if (filename != ".DS_Store"):
            
            id_p = filename[0:4]
            ID_Graph.append(id_p)
            matrix = "../../R/Rdati/Matrici Adiacenza/{}.csv".format(id_p)            
            proteina = pd.read_csv(matrix, sep = ';')
            Amino.extend(list(proteina.iloc[0][1:]))


Amino_TOT = list(OrderedDict.fromkeys(Amino))

classes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']

if not exists("../Dati/aminoacidi.pickle"):
    aminoAcidi = preprocessing.LabelEncoder()
    aminoAcidi.fit(Amino_TOT)
    f = open("../Dati/aminoacidi.pickle","wb")
    pickle.dump(aminoAcidi, f)
    f.close()

if not exists("../Dati/classi.pickle"):
    classi = preprocessing.LabelEncoder()
    classi.fit(classes)
    f = open("../Dati/classi.pickle","wb")
    pickle.dump(classi, f)
    f.close()

if not exists("../Dati/ID_Graphs.pickle"):
    ID_Graphs = preprocessing.LabelEncoder()
    ID_Graphs.fit(ID_Graph)
    f = open("../Dati/ID_Graphs.pickle","wb")
    pickle.dump(ID_Graphs, f)
    f.close()

In [46]:
proteine_path = "../Dati/proteine.csv"
proteine = pd.read_csv(proteine_path, sep = ',')

f = open("../Dati/classi.pickle","rb")
classi = pickle.load(f)
f.close()

f = open("../Dati/aminoacidi.pickle","rb")
amino = pickle.load(f)
f.close()

f = open("../Dati/ID_Graphs.pickle","rb")
ID_Graphs_read = pickle.load(f)
f.close()

ID_Graph = []

edges = pd.DataFrame(columns=["graph_id", "src_id", "dst_id"])

nodes = pd.DataFrame(columns=["graph_id", "node_id", "label"])

graphs = pd.DataFrame(columns=["graph_id", "label"])


for filename in os.listdir("../../R/Rdati/Matrici Adiacenza/"):
    with open(os.path.join("../../R/Rdati/Matrici Adiacenza/", filename), 'r', encoding='windows-1252') as f:
        if (filename != ".DS_Store"):
            id_p = filename[0:4]
            print(id_p)

            #Id della proteina che si sta analizzando
            #print(id_p)

            if id_p not in ID_Graph: 
                ID_Graph.append(id_p)
                matrix = "../../R/Rdati/Matrici Adiacenza/{}.csv".format(id_p)
                #print(matrix)

                #Matrice adiacenza singola matrice inserita in un dataframe
                proteina = pd.read_csv(matrix, sep = ';')
                #Nomi dei nodi nonché numero
                Nodes = list(proteina.columns[1:])
                
                for vertex in range(0,len(Nodes)):   
                    Nodes[vertex] = Nodes[vertex][1:]
                    
                #Lista degli Amino acidi
                    #Amino = list(proteina.iloc[0][1:])
                Amino = amino.transform(list(proteina.iloc[0][1:]))

                
                proteina.drop(proteina.columns[[0]], axis = 1, inplace = True)
                j = proteina.iloc[0]
                proteina.drop([0], axis = 0, inplace = True)
                
                
                    #proteina.drop(proteina.columns[[0]], axis = 1, inplace = True)
                    #proteina.drop([0], axis = 0, inplace = True)
                #Classe della proteina
                classe = proteine[proteine['Id'] == id_p]["Classe"].iloc[0][0]
        
                
                #Creazione Grafo
                G = nx.Graph()
                G.add_nodes_from(Nodes)
                
                #Aggiunta archi
                rows, cols = proteina.shape
                Vprimo = "Vx"
                Vsecondo ="Vx"
                for i in range(rows):
                    row = proteina.iloc[i]
                    j = 0
                    for edge in row:
                        if edge == '1':
                            vPrimo = Vprimo[:1] + str(i+1)
                            vSecondo = Vsecondo[:1] + str(j+1)
                            #print("Aggiunta arco tra il nodo ", vPrimo[1:],"e il nodo ", vSecondo[1:])
                            G.add_edge(vPrimo[1:], vSecondo[1:])
                        j += 1
                
                #Conbersione Grafo in formato DGL
                new_g = dgl.from_networkx(G)
                
                
                
                #Creazione file CSV Grafi
                id_graph = ID_Graphs_read.transform([id_p])[0]

                a = new_g.edges()[0]
                b = new_g.edges()[1]
                src_id = a.tolist()
                dst_id = b.tolist()
                    
                
                graph_id = []
                for i in range(0,len(src_id)):
                    graph_id.append(id_graph)
                
                # Create dict edge
                data = {'graph_id': graph_id,
                        'src_id': src_id,
                        'dst_id': dst_id
                       }

                # Create DataFrame
                edge = pd.DataFrame(data)
                
                edges = edges.merge(edge, how='outer')
                
                
                node = new_g.nodes()
                node_id = node.tolist()
                graph_id = graph_id[:len(Amino)]

                
                Amino = Amino.astype("float")
                # Create dict node
                data = {'graph_id': graph_id,
                        'node_id': node_id,
                        'label': Amino
                       }

                # Create DataFrame
                node = pd.DataFrame(data)
 
                nodes = nodes.merge(node, how = 'outer')
    
                
                classe = classi.transform([classe])
                graph_id = graph_id[:1]

                
                classe = classe.astype("float")
                # Create dict graph
                data = {'graph_id': graph_id,
                        'label': classe
                       }

                # Create DataFrame
                graph = pd.DataFrame(data)
                
                graphs = graphs.merge(graph, how = 'outer')
                
edges.to_csv('../Dati/Dataset/edges.csv', index=False)
nodes.to_csv('../Dati/Dataset/nodes.csv', index=False)
graphs.to_csv('../Dati/Dataset/graphs.csv', index=False)

4vhb
1ri8
1cdu
1cdb
1jtp
1cid
1dly
1ash
1s61
1cdc
1rjc
1ch4
1ci5
2atp
1s6a
1ncn
1uvx
1ieh
3cd4
2gl3
1op9
1mel
1jtt
1f5w
1ut0
1uvy
1urv
1g9e
5yxk
1sjv
4max
2giy
1eaj
3tm3
2vhb
1zmy
1rte
1i8l
1wio
3tld
2hz1
1v5h
1zv5
1dr9
2hz2
2hz3
2gkn
3tm9
1zvy
4l2m
1ccz
5v3v
1rtx
2gkm
5v3t
1a7b
2dc3
1a6p
5v3u
1ngk
1ury
1wiq
2ksc
4hsw
1qd0
1a64
1zvh
1gya
1wip
2bmm
1mwb
1mvf
1i85
3o2d
3vhb
1s69
1jto
2aw2
1qa9
1cdj
1i3u
1cdh
1s56
2qrw
1cdi
1i3v
1ux8
3ag0
1l6z
1umo
1dlw
2gln
1ux9
1idr
1cdy
1cd8
3lb2
3b9k
1hnf
1vhb
1hcv
1xfp
1hng
2bkm


In [47]:
graphs

,graph_id,label
0,99,0.0
1,44,1.0
2,11,1.0
3,6,1.0
4,33,1.0
...,...,...
99,62,0.0
100,23,1.0
101,66,1.0
102,25,1.0


In [44]:
edges

,graph_id,src_id,dst_id
0,99,0,50
1,99,0,61
2,99,0,72
3,99,0,83
4,99,0,94
...,...,...,...
458397,73,31,30
458398,73,31,32
458399,73,32,29
458400,73,32,30


In [45]:
nodes

,graph_id,node_id,label
0,99,0,12.0
1,99,1,10.0
2,99,2,3.0
3,99,3,5.0
4,99,4,5.0
...,...,...,...
14683,73,123,14.0
14684,73,124,3.0
14685,73,125,8.0
14686,73,126,10.0
